In [1]:
import scrapy
from scrapy.http import HtmlResponse
from scrapy import Selector

import requests

In [2]:
class ImdbSpider(scrapy.Spider):
    name = "imdb"
    allowed_domains = ["imdb.com"]
    start_urls = ['http://www.imdb.com/chart/top',]
   
    def parse(self, response):
        # получение таблицы со строками, хараактеризующими ТОП-фильмы
        table_rows = response.xpath(
            './/*[@class="chart full-width" and @data-caller-name="chart-top250movie"]/'
            'tbody[@class="lister-list"]/tr'
        
        )
        
        for row in table_rows:
            # для каждой строки извлечем необходимую информацию
            yield {
                # пока парсим 3 колонки
                "title": row.xpath("./td[@class='titleColumn']/a/text()").extract_first(),
                "year": row.xpath("./td[@class='titleColumn']/span/text()").extract_first().strip("() "),
                "rating": row.xpath("./td[@class='ratingColumn imdbRating']/strong/text()").extract_first(),
            }

In [3]:
DEBUG_URL = 'http://www.imdb.com/chart/top'

req = requests.get(DEBUG_URL)
response = HtmlResponse(url=DEBUG_URL, body=req.content)

selector = Selector(response=response)

In [4]:
selector.xpath('.//*[@class="chart full-width" and @data-caller-name="chart-top250movie"]'
               '/tbody[@class="lister-list"]')[0].extract()

'<tbody class="lister-list">\n\n  <tr>\n    <td class="posterColumn">\n\n    <span name="rk" data-value="1"></span>\n    <span name="ir" data-value="9.219810621198208"></span>\n    <span name="us" data-value="7.791552E11"></span>\n    <span name="nv" data-value="2400711"></span>\n    <span name="ur" data-value="-1.7801893788017917"></span>\n<a href="/title/tt0111161/"> <img src="https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UY67_CR0,0,45,67_AL_.jpg" width="45" height="67" alt="Побег из Шоушенка">\n</a>    </td>\n    <td class="titleColumn">\n      1.\n      <a href="/title/tt0111161/" title="Frank Darabont (dir.), Tim Robbins, Morgan Freeman">Побег из Шоушенка</a>\n        <span class="secondaryInfo">(1994)</span>\n    </td>\n    <td class="ratingColumn imdbRating">\n            <strong title="9.2 based on 2,400,711 user ratings">9.2</strong>\n    </td>\n    <td class="ratingColumn">\n    <div class="seen-widget

In [5]:
selector.xpath('.//*[@class="chart full-width" and @data-caller-name="chart-top250movie"]'
               '/tbody[@class="lister-list"]/tr')[0].extract()

'<tr>\n    <td class="posterColumn">\n\n    <span name="rk" data-value="1"></span>\n    <span name="ir" data-value="9.219810621198208"></span>\n    <span name="us" data-value="7.791552E11"></span>\n    <span name="nv" data-value="2400711"></span>\n    <span name="ur" data-value="-1.7801893788017917"></span>\n<a href="/title/tt0111161/"> <img src="https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UY67_CR0,0,45,67_AL_.jpg" width="45" height="67" alt="Побег из Шоушенка">\n</a>    </td>\n    <td class="titleColumn">\n      1.\n      <a href="/title/tt0111161/" title="Frank Darabont (dir.), Tim Robbins, Morgan Freeman">Побег из Шоушенка</a>\n        <span class="secondaryInfo">(1994)</span>\n    </td>\n    <td class="ratingColumn imdbRating">\n            <strong title="9.2 based on 2,400,711 user ratings">9.2</strong>\n    </td>\n    <td class="ratingColumn">\n    <div class="seen-widget seen-widget-tt0111161 pending" d

In [6]:
from scrapy.crawler import CrawlerProcess

In [7]:
process = CrawlerProcess()

process.crawl(ImdbSpider)
process.start()

2021-06-01 17:02:50 [scrapy.utils.log] INFO: Scrapy 2.5.0 started (bot: scrapybot)
2021-06-01 17:02:50 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.7.10 (default, May  3 2021, 02:48:31) - [GCC 7.5.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1k  25 Mar 2021), cryptography 3.4.7, Platform Linux-5.4.72-microsoft-standard-WSL2-x86_64-with-Ubuntu-18.04-bionic
2021-06-01 17:02:50 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-06-01 17:02:50 [scrapy.crawler] INFO: Overridden settings:
{}
2021-06-01 17:02:50 [scrapy.extensions.telnet] INFO: Telnet Password: a1d27c720fe9272a
2021-06-01 17:02:50 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2021-06-01 17:02:50 [scrapy.middleware] INFO: Enabled downloader middle

## Пример с обходом страниц

In [8]:
import scrapy
from scrapy.http import HtmlResponse
from scrapy import Selector

import requests

In [ ]:
# dataclass

class MovieItem(scrapy.Item):
    title = scrapy.Field()
    rating = scrapy.Field()
    summary = scrapy.Field()
    genre = scrapy.Field()
    runtime = scrapy.Field()
    directors = scrapy.Field()
    writers = scrapy.Field()
    cast = scrapy.Field()

class ActorItem(scrapy.Item):
    actor_name = scrapy.Field()
    character = scrapy.Field()

In [ ]:
class ImdbSpider(scrapy.Spider):
    name = "imdb"
    allowed_domains = ["imdb.com"]
    base_url = "https://imdb.com"
    start_urls = ['https://www.imdb.com/chart/top',]
   
    def parse(self, response):
        table_rows = response.xpath(
            './/*[@class="chart full-width" and @data-caller-name="chart-top250movie"]/'
            'tbody[@class="lister-list"]/tr'
        
        )
        for row in table_rows:
            rating = row.xpath("./td[@class='ratingColumn imdbRating']/strong/text()").extract_first(),
            rel_url = row.xpath("td[@class='titleColumn']/a/@href").extract_first().strip()
            row_url = self.base_url + rel_url
            yield scrapy.Request(row_url, callback=self.parseOneMovie, meta={'rating' : rating})
    
    def parseOneMovie(self, response):
        item = MovieItem()
        item["rating"] = response.meta["rating"][0]
        item['title'] = response.xpath('.//*/div[@class="title_wrapper"]/h1/text()').extract_first().strip()
        item["summary"] = response.xpath(".//*/div[@class='summary_text']/text()").extract_first().strip()
        item['directors'] = response.xpath('.//*/div[@class="credit_summary_item"]/a/text()').extract_first().strip()
        item['writers'] = response.xpath('.//*/div[@class="credit_summary_item"]/a/text()').extract()[1].strip()

        item["cast"] = list()

        for cast in response.xpath(".//table[@class='cast_list']/tr")[1:]:
            actor = ActorItem()
            
            actor["actor_name"] = cast.xpath("./td[2]/a/text()").extract_first().strip()
            actor["character"] = cast.xpath("./td[@class='character']/a/text()").extract_first()
            
            item["cast"].append(actor)

        return item

In [ ]:
from scrapy.crawler import CrawlerProcess

In [ ]:
process = CrawlerProcess()

process.crawl(ImdbSpider)
process.start()

In [ ]:
DEBUG_URL = 'http://www.imdb.com/title/tt0019254/'

req = requests.get(DEBUG_URL)
response = HtmlResponse(url=DEBUG_URL, body=req.content)

selector = Selector(response=response)


In [ ]:
selector.xpath(".//table[@class='cast_list']/tr")